![](/files/programmingwallet.png)


![](/files/session3/s3.png)


# Session 3 Objectives
* Learn HD Wallets (BIP32)
* Learn HD Wallet Organization (BIP44)
* Learn Mnemonic Backup (BIP39)
* Create Wallet Seeds


# HD Wallets


# Motivation
* Reusing Addresses compromises privacy
* Creating a new private key means having to back it up
* Core wallet used to generate 100 private keys at a time
* Backing up many private keys is not easy to do on paper


# Deterministic Wallets
* Single seed can generate virtually infinite private keys
* (N+1)st private key generated from the Nth private key and a code
* Used first in Armory back in 2012


# Naive Deterministic Wallet
* $eG=P$
* $(e+1)G=eG+G=P+G$
* $(e+2)G=eG+2G=P+2G$
* ...
* Store private key $e$ and reveal $P$ to payer
* No privacy from chain analysis or payer


# Robust Deterministic Wallet
* $eG=P$, $c$ is a shared secret, $H$ is a hash function
* $(e+H(c,P,1))G=P+H(1+P,c)G$
* $(e+H(c,P,2))G=P+H(2+P,c)G$
* ...
* Store private key $e$ and reveal $P$, $c$ to payer
* Privacy from chain analysis
* No privacy from payers


# Message Authentication Codes
* Used for verifying that a message is authentic when you share a secret already
* HMAC is an implementation of MAC where H stands for "hash-based"
* Most cryptographic hash functions (like sha256) have an HMAC implementation


# Heirarchical Deterministic Wallets (BIP32)
* Single seed + chain code (shared secret)
* The master private key can generate $2^{32}$ child private keys
* Every child private key can also generate $2^{32}$ child private keys
* Revealing a child public key does not reveal the parent public key
* Adds privacy from recpients


# Implementation
* $eG=P$, $e$ is the private key, $P$ is the public key
* $c$ is the chaincode $H$ is the HMAC-SHA512 function
* $2^{31}$ hardened keys and $2^{31}$ unhardened keys
* Hardened means the public key ($P$) and chain code ($c$) *cannot* derive the child public key.
* Unhardened means the public key ($P$) and chain code ($c$) *can* derive the child public key


# Defining a HD Public Key
* public key - normal ECC public key (33 bytes)
* chain code - 32-byte shared secret with payers
* depth - 0 is root, 1 is a child of root, 2 is grandchild of root, etc.
* fingerprint- `00000000` for root, first 4 bytes of parent pubkey's hash160.
* child number - ordering from parent


# Defining a HD Private Key
* private key - normal ECC private key (33 bytes)
* chain code - 32-byte shared secret with payers
* depth - 0 is root, 1 is a child of root, 2 is grandchild of root, etc.
* fingerprint- `00000000` for root, first 4 bytes of parent pubkey's hash160.
* child number - ordering from parent


# Process for generating a Master HD Private Key
* Create a seed of 128 to 512 bits
* Calculate $h=H(d,s)$ where $H$ is HMAC-SHA512, $d$ is `Bitcoin seed` and $s$ is the seed.
* Master secret = first 256 bits of $h$ in big-endian
* Master chain code = last 256 bits of $h$ in big-endian


In [ ]:
# Example Master Key Generation
from ecc import PrivateKey
from helper import big_endian_to_int, hmac_sha512, raw_decode_base58
from hd import HDPrivateKey
seed = b'jimmy@programmingblockchain.com Jimmy Song'
h = hmac_sha512(b'Bitcoin seed', seed)
private_key = PrivateKey(secret=big_endian_to_int(h[:32]))
chain_code = h[32:]
master = HDPrivateKey(
    private_key=private_key,
    chain_code=chain_code,
    testnet=True,
)
print(master.bech32_address())

In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import hd, tx

### Exercise 1




#### Make [this test](/edit/session3/hd.py) pass: `hd.py:HDTest:test_from_seed`

In [ ]:
# Exercise 1

reload(hd)
run(hd.HDTest('test_from_seed'))

In [ ]:
# Example Unhardened Child Derivation
from ecc import N
from hd import HDPrivateKey
from helper import big_endian_to_int, hmac_sha512, int_to_big_endian
seed_phrase = b'jimmy@programmingblockchain.com Jimmy Song'
master = HDPrivateKey.from_seed(seed_phrase, True)
index = 0
data = master.private_key.point.sec() + int_to_big_endian(index, 4)
h = hmac_sha512(master.chain_code, data)
secret = (big_endian_to_int(h[:32]) + master.private_key.secret) % N
unhardened_child = HDPrivateKey(
    private_key=PrivateKey(secret=secret),
    chain_code=h[32:],
    depth=master.depth + 1,
    parent_fingerprint=master.fingerprint(),
    child_number=index,
    testnet=master.testnet,
)
print(unhardened_child.bech32_address())

In [ ]:
# Example Hardened Child Derivation
from ecc import N
from hd import HDPrivateKey
from helper import big_endian_to_int, hmac_sha512, int_to_big_endian
seed_phrase = b'jimmy@programmingblockchain.com Jimmy Song'
master = HDPrivateKey.from_seed(seed_phrase, True)
index = 0x80000002
data = int_to_big_endian(master.private_key.secret, 33) + int_to_big_endian(index, 4)
h = hmac_sha512(master.chain_code, data)
secret = (big_endian_to_int(h[:32]) + master.private_key.secret) % N
hardened_child = HDPrivateKey(
    private_key=PrivateKey(secret=secret),
    chain_code=h[32:],
    depth=master.depth + 1,
    parent_fingerprint=master.fingerprint(),
    child_number=index,
    testnet=master.testnet,
)
print(hardened_child.bech32_address())

### Exercise 2




#### Make [this test](/edit/session3/hd.py) pass: `hd.py:HDTest:test_child`

In [ ]:
# Exercise 2

reload(hd)
run(hd.HDTest('test_child'))

In [ ]:
# example of private key path traversal
from hd import HDPrivateKey
seed_phrase = b'jimmy@programmingblockchain.com Jimmy Song'
master = HDPrivateKey.from_seed(seed_phrase, True)
current = master
path = "m/0/1'/2/3'"
components = path.split('/')[1:]
for child in components:
    if child.endswith("'"):
        index = int(child[:-1]) + 0x80000000
    else:
        index = int(child)
    current = current.child(index)
print(current.bech32_address())

### Exercise 3




#### Make [this test](/edit/session3/hd.py) pass: `hd.py:HDTest:test_traverse`

In [ ]:
# Exercise 3

reload(hd)
run(hd.HDTest('test_traverse'))

In [ ]:
# Example to create an xpub
from hd import HDPrivateKey
from helper import encode_base58_checksum, int_to_byte, int_to_big_endian
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
hd_priv = HDPrivateKey.from_seed(passphrase)
raw = bytes.fromhex('0488b21e')
raw += int_to_byte(hd_priv.depth)
raw += hd_priv.parent_fingerprint
raw += int_to_big_endian(hd_priv.child_number, 4)
raw += hd_priv.chain_code
raw += hd_priv.pub.point.sec()
print(encode_base58_checksum(raw))

### Exercise 4




#### Make [this test](/edit/session3/hd.py) pass: `hd.py:HDTest:test_prv_pub`

In [ ]:
# Exercise 4

reload(hd)
run(hd.HDTest('test_prv_pub'))

### Exercise 5




#### Make [this test](/edit/session3/hd.py) pass: `hd.py:HDTest:test_parse`

In [ ]:
# Exercise 5

reload(hd)
run(hd.HDTest('test_parse'))

### Exercise 6

#### Create an extended public key

Create a xpub on testnet (should start with tpub)



In [ ]:
# Exercise 6

from hd import HDPrivateKey
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
# create an HDPrivateKey instance using from_seed on testnet
hd_priv = HDPrivateKey.from_seed(passphrase, testnet=True)
# print the xpub
print(hd_priv.xpub())

In [ ]:
# Example of getting p2pkh/p2sh-p2wpkh/p2wpkh testnet addresses
from hd import HDPrivateKey
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
hd_priv = HDPrivateKey.from_seed(passphrase, testnet=True)
# p2pkh
p2pkh_path = "m/44'/1'/0'/0/0"
print(hd_priv.traverse(p2pkh_path).address())
# p2sh-p2wpkh
p2sh_p2wpkh_path = "m/49'/1'/0'/0/0"
print(hd_priv.traverse(p2sh_p2wpkh_path).p2sh_p2wpkh_address())
# p2wpkh
p2wpkh_path = "m/84'/1'/0'/0/0"
print(hd_priv.traverse(p2wpkh_path).bech32_address())

### Exercise 7




#### Make [this test](/edit/session3/hd.py) pass: `hd.py:HDTest:test_get_address`

In [ ]:
# Exercise 7

reload(hd)
run(hd.HDTest('test_get_address'))

### Exercise 8

#### Create external p2pkh, p2sh_p2wpkh and p2wpkh addresses



In [ ]:
# Exercise 8

from hd import HDPrivateKey
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
# create an HDPrivateKey instance using from_seed on testnet
hd_priv = HDPrivateKey.from_seed(passphrase, testnet=True)
# print the p2pkh address
print(hd_priv.get_p2pkh_receiving_address())
# print the p2sh-pwpkh address
print(hd_priv.get_p2sh_p2wpkh_receiving_address())
# print the p2wpkh address
print(hd_priv.get_p2wpkh_receiving_address())

### Exercise 9

#### Create xpub for account 0



In [ ]:
# Exercise 9

from hd import HDPrivateKey
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
# create an HDPrivateKey instance using from_seed on testnet
hd_priv = HDPrivateKey.from_seed(passphrase, testnet=True)
# calculate the path for purpose=44', coin=1' (testnet), account=0
path = "m/44'/1'/0'"
# print the xpub at that path
print(hd_priv.traverse(path).xpub())

In [ ]:
def secure_mnemonic(entropy=0, num_bits=128):
    # if we have more than 128 bits, just mask everything but the last 128 bits
    if len(bin(entropy)) > num_bits+2:
        entropy &= (1 << num_bits) - 1
    # xor some random bits with the entropy that was passed in
    preseed = randbits(num_bits) ^ entropy
    # convert the number to big-endian
    s = int_to_big_endian(preseed, 16)
    # 1 extra bit for checksum is needed per 32 bits
    checksum_bits_needed = num_bits // 32
    # the checksum is the sha256's first n bits. At most this is 8
    checksum = sha256(s)[0] >> (8 - checksum_bits_needed)
    # we concatenate the checksum to the preseed
    total = (preseed << checksum_bits_needed) | checksum
    # now we get the mnemonic passphrase
    mnemonic = []
    # now group into groups of 11 bits
    for _ in range((num_bits + bits_needed) // 11):
        # grab the last 11 bits
        current = total & ((1 << 11) - 1)
        # insert the correct word at the front
        mnemonic.insert(0, WORD_LIST[current])
        # shift by 11 bits so we can move to the next set
        total >>= 11
    # return the mnemonic phrase by putting spaces between
    return ' '.join(mnemonic)

In [ ]:
from hd import HDPrivateKey
from helper import hmac_sha512_kdf, sha256
from mnemonic import WORD_LOOKUP, WORD_LIST
mnemonic = 'legal winner thank year wave sausage worth useful legal winner thank yellow'
password = b'TREZOR'
words = mnemonic.split()
if len(words) not in (12, 15, 18, 21, 24):
    raise ValueError('you need 12, 15, 18, 21, or 24 words')
number = 0
for word in words:
    index = WORD_LOOKUP[word]
    number = (number << 11) | index
checksum_bits_length = len(words) // 3
checksum = number & ((1 << checksum_bits_length) - 1)
data_num = number >> checksum_bits_length
data = int_to_big_endian(data_num, checksum_bits_length * 4)
computed_checksum = sha256(data)[0] >> (8 - checksum_bits_length)
if checksum != computed_checksum:
    raise ValueError('words fail checksum: {}'.format(words))
normalized_words = []
for word in words:
    normalized_words.append(WORD_LIST[WORD_LOOKUP[word]])
normalized_mnemonic = ' '.join(normalized_words)
salt = b'mnemonic' + password
seed = hmac_sha512_kdf(normalized_mnemonic, salt)
print(HDPrivateKey.from_seed(seed).xprv())

### Exercise 10




#### Make [this test](/edit/session3/hd.py) pass: `hd.py:HDTest:test_from_mnemonic`

In [ ]:
# Exercise 10

reload(hd)
run(hd.HDTest('test_from_mnemonic'))

### Exercise 11

#### Generate a testnet extended public key at m/84'/1'/0' using the generic mnemonic and your own passphrase



In [ ]:
# Exercise 11

from hd import HDPrivateKey
mnemonic = 'abandon abandon abandon abandon abandon abandon abandon abandon abandon abandon abandon about'
passphrase = b'jimmy@programmingblockchain.com Jimmy Song'
path = "m/84'/1'/0'"
# create a private key using the mnemonic, passphrase, path and testnet=True
hd_priv = HDPrivateKey.from_mnemonic(mnemonic, passphrase, path, True)
# print the xpub
print(hd_priv.xpub())